In [1]:
import requests
import pandas as pd

# Function to fetch all available location categories
def get_location_categories(token):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/locationcategories"
    headers = {
        'token': token
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()['results']
    else:
        print("Error fetching location categories:")
        print(response.text)  # Print response content for debugging
        return None

# Function to fetch locations for a given location category
def get_locations_by_category(token, category_id):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/locations"
    headers = {
        'token': token
    }
    params = {
        'locationcategoryid': category_id,
        'limit': 1000  # Increase if necessary
    }

    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()['results']
    else:
        print("Error fetching locations:")
        print(response.text)  # Print response content for debugging
        return None

# Function to filter locations by country code
def filter_locations_by_country(locations, country_code):
    filtered_locations = []
    for location in locations:
        # Extract country code from location name
        location_country_code = location['name'].split(',')[-1].strip()
        if location_country_code == country_code:
            filtered_locations.append(location)
    return filtered_locations

# Function to fetch GSOY data for a given location and date range
def get_gsoy_data(token, location_id, start_date, end_date):
    url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
    headers = {
        'token': token
    }
    params = {
        'datasetid': 'GSOY',  # Global Summary of the Year
        'locationid': location_id,
        'startdate': start_date,
        'enddate': end_date,
        'units': 'metric',  # Use 'standard' for Fahrenheit, inches, etc.
        'limit': 1000  # Adjust as needed
    }

    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()['results']
    else:
        print("Error fetching GSOY data:")
        print(response.text)  # Print response content for debugging
        return None

# Example usage
token = 'pQEtCzeZGBUnHfKnnUDFOVpcMDpHRwHc'
country_code = 'FR'  # Example country code for the United States
start_date = '2020-01-01'
end_date = '2020-12-31'

# Fetch all available location categories
categories = get_location_categories(token)
if categories:
    for category in categories:
        print(f"Fetching locations for category: {category['name']}")
        # Fetch locations for the current category
        locations = get_locations_by_category(token, category['id'])
        if locations:
            # Filter locations by country code
            country_locations = filter_locations_by_country(locations, country_code)
            if country_locations:
                for location in country_locations:
                    print(f"Fetching GSOY data for location: {location['name']}")
                    # Fetch GSOY data for the current location
                    gsoy_data = get_gsoy_data(token, location['id'], start_date, end_date)
                    if gsoy_data:
                        # Convert data to DataFrame
                        df = pd.DataFrame(gsoy_data)
                        # Save DataFrame to CSV
                        file_name = f"{country_code}_{category['id']}_{location['id']}_gsoy_data.csv"
                        df.to_csv(file_name, index=False)
                        print(f"Data saved to {file_name}")
                    else:
                        print("No GSOY data found for the specified location and date range")
            else:
                print("No locations found for the specified country code")
        else:
            print("No locations found for the current category")
else:
    print("No location categories found")


Fetching locations for category: City
Fetching GSOY data for location: Ajaccio, FR
Data saved to FR_CITY_CITY:FR000001_gsoy_data.csv
Fetching GSOY data for location: Amiens, FR


KeyError: 'results'